In [2]:
import pandas as pd

import tqdm

import random

import copy

In [3]:
class DictList(dict):
    def __setitem__(self, key, value):
        try:
            # Assumes there is a list on the key
            self[key].append(value)
        except KeyError: # If it fails, because there is no key
            super(DictList, self).__setitem__(key, value)
        except AttributeError: # If it fails because it is not a list
            super(DictList, self).__setitem__(key, [self[key], value])

In [4]:
def update_sets(sets_, covered):
    
    sets_ = [s.difference(covered) for s in sets_]
    
    return sets_

In [5]:
df = pd.read_csv("sets_per_strategy.csv")

In [6]:
original_sets_ = []

for i, row in tqdm.tqdm_notebook(df.iterrows()):
    
    original_sets_.append(set(row.dropna().values))

<ipython-input-6-6e43ec315a69>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, row in tqdm.tqdm_notebook(df.iterrows()):


In [7]:
sets_ = copy.deepcopy(original_sets_)

sizes = DictList()

for i, s in enumerate(sets_):
    sizes[len(s)] = i

In [11]:
parameters = []

for key in sizes.keys():
    parameters.append((key, len(sizes[key])))

In [14]:
parameters.sort(key=lambda x: x[0])

In [16]:
parameters[::-1]

[(51, 36),
 (50, 97),
 (49, 35),
 (48, 37),
 (47, 25),
 (46, 42),
 (45, 32),
 (44, 4),
 (43, 6),
 (42, 83),
 (41, 54),
 (40, 60),
 (39, 24),
 (38, 77),
 (37, 34),
 (36, 51),
 (35, 83),
 (34, 79),
 (33, 72),
 (32, 68),
 (31, 221),
 (30, 232),
 (29, 415),
 (28, 200),
 (27, 210),
 (26, 189),
 (25, 214),
 (24, 259),
 (23, 246),
 (22, 282),
 (21, 329),
 (20, 300),
 (19, 393),
 (18, 349),
 (17, 348),
 (16, 791),
 (15, 990),
 (14, 720),
 (13, 1212),
 (12, 1700),
 (11, 1537),
 (10, 2463),
 (9, 2238),
 (8, 3053),
 (7, 2563),
 (6, 2028),
 (5, 2125),
 (4, 2016),
 (3, 2721),
 (2, 3049),
 (1, 1374)]

In [115]:
len(sizes[50])

97

In [116]:
len(sizes[49])

35

In [117]:
len(sizes[48])

37

In [118]:
seeds = range(10)

In [119]:
import itertools

In [130]:
parameters_sweep = (itertools.product([51], range(36), seeds))

In [141]:
import itertools

parameters_sweep = (itertools.product([48], range(37), seeds))


for starting_size, starting_index, seed in tqdm.notebook.tqdm(parameters_sweep):


    ### Initialisation ###

    sets_ = copy.deepcopy(original_sets_)

    sizes = DictList()

    for i, s in enumerate(sets_):
        sizes[len(s)] = i

    random.seed(seed)

    universe = {i for i in range(94)}

    covered = set()

    indices = []

    ### First Iteration ###

    if starting_index:
        index = sizes[starting_size][starting_index]
        covered |= sets_[index]
        sets_.pop(index)
        indices.append(index)
    else:
        index = random.choice(sizes[starting_size])
        covered |= sets_[index]
        sets_.pop(index)
        indices.append(index)

    sets_ = update_sets(sets_, covered)


    ### Main body ###

    while covered != universe:

        sizes = DictList()

        for i, s in enumerate(sets_):
            sizes[len(s)] = i

        max_coverage = max(sizes.keys())

        if isinstance(sizes[max_coverage], list):
            index = random.choice(sizes[max_coverage])
        else:
            index = sizes[max_coverage]

        subset = sets_[index]

        covered |= subset

        indices.append(index)

        sets_.pop(index)

        sets_ = update_sets(sets_, covered)
        
    with open(f'../data/set_cover_output/output_starting_size_{starting_size}_starting_index_{starting_index}_seed_{seed}.txt', 'w') as f:
        data = [starting_index, len(indices), *indices]
                 
        f.write(",".join([str(d) for d in data]))

In [4]:
import glob

import tqdm

In [20]:
files = glob.glob("../data/set_cover_output/*.txt")

In [21]:
len(files)

68980

In [22]:
with open(file) as f:

    lines = f.readlines()

In [23]:
lines

['331,12,28443,24666,2459,7748,1329,14215,4278,31705,26404,4468,12239,8921']

In [18]:
smallest_sets = []

info = {}

count = 0

for file in tqdm.tqdm_notebook(files):
    with open(file) as f:

        lines = f.readlines()
        
#         if int(lines[0].split(",")[1]) == 10:

        seed = int(file.split("seed_")[-1].split(".txt")[0])

        starting_size = int(file.split("starting_size_")[-1].split("_")[0])

        starting_index = int(file.split("starting_index_")[-1].split("_")[0])


        info[count] = {"seed": seed, 
                       "starting_size": starting_size,
                       "starting_index": starting_index,
                       "indices": lines[0].split(",")[2:],
                       "len": len(lines[0].split(",")[2:])}


        count += 1

<ipython-input-18-1ff89b22bd03>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm.tqdm_notebook(files):


In [19]:
import collections

collections.Counter([k['len'] for k in info.values()])

Counter({13: 19856, 12: 28385, 14: 3899, 11: 14876, 10: 1964})

In [13]:
smallest_sets = [len(k['indices']) for k in info.values()]

In [15]:
min(smallest_sets)

10

In [16]:
len(smallest_sets), len(files)

(1964, 68980)

In [164]:
min(smallest_sets)

10

In [196]:
file.split("starting_index_")[-1].split("_")[0]

'1'

In [166]:
smallest_sets.sort()

In [168]:
import collections

In [169]:
collections.Counter(smallest_sets)

Counter({10: 259, 11: 932, 12: 699, 13: 160})

In [174]:
seed = int(file.split("seed_")[-1].split(".txt")[0])

starting_size = int(file.split("starting_size_")[-1].split("_")[0])

'2'

In [205]:
indices = [i['indices'] for i in info.values() if i['starting_size'] == 50]

In [206]:
indices

[['14001',
  '20407',
  '28507',
  '18244',
  '28204',
  '30941',
  '148',
  '14876',
  '5945',
  '8987'],
 ['22930',
  '22582',
  '7167',
  '25956',
  '24462',
  '16227',
  '182',
  '10230',
  '13221',
  '1625'],
 ['28979',
  '20408',
  '28508',
  '18245',
  '28205',
  '30941',
  '148',
  '14877',
  '5945',
  '8987'],
 ['26700',
  '22582',
  '7167',
  '25957',
  '24463',
  '16227',
  '182',
  '10230',
  '13221',
  '1625'],
 ['10866',
  '24464',
  '27903',
  '1805',
  '17170',
  '30941',
  '14476',
  '27192',
  '35643',
  '13090'],
 ['12042',
  '24464',
  '27903',
  '1805',
  '17170',
  '30941',
  '14476',
  '27192',
  '35643',
  '13090'],
 ['13021',
  '24464',
  '27903',
  '1805',
  '17170',
  '30941',
  '14476',
  '27192',
  '35643',
  '13090'],
 ['29072',
  '24089',
  '27904',
  '1805',
  '17171',
  '30941',
  '14477',
  '27193',
  '35643',
  '13091'],
 ['32842',
  '4564',
  '4713',
  '1329',
  '10618',
  '30649',
  '1248',
  '14986',
  '35141',
  '13090'],
 ['33028',
  '20408',
  '